In [1]:
! pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j
! pip install --upgrade --quiet langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.

In [3]:
NEO4J_URI="YOUR_NEO4J_URL"
NEO4J_USERNAME="YOUR_NEO4J_USERNAME"
NEO4J_PASSWORD="YOUR_NEO4J_PASSWORD"
GROQ_API_KEY="YOUR_GROQ_API_KEY"

In [4]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [5]:
import pandas as pd
df = pd.read_csv('/content/amazon_prime_users.csv')
column_names = df.columns.tolist()


In [6]:
print(column_names)


['User ID', 'Name', 'Email Address', 'Username', 'Date of Birth', 'Gender', 'Location', 'Membership Start Date', 'Membership End Date', 'Subscription Plan', 'Payment Information', 'Renewal Status', 'Usage Frequency', 'Purchase History', 'Favorite Genres', 'Devices Used', 'Engagement Metrics', 'Feedback/Ratings', 'Customer Support Interactions']


In [7]:
df.head()

,User ID,Name,Email Address,Username,Date of Birth,Gender,Location,Membership Start Date,Membership End Date,Subscription Plan,Payment Information,Renewal Status,Usage Frequency,Purchase History,Favorite Genres,Devices Used,Engagement Metrics,Feedback/Ratings,Customer Support Interactions
0,1,Ronald Murphy,williamholland@example.com,williamholland,1953-06-03,Male,Rebeccachester,2024-01-15,2025-01-14,Annual,Mastercard,Manual,Regular,Electronics,Documentary,Smart TV,Medium,3.6,3
1,2,Scott Allen,scott22@example.org,scott22,1978-07-08,Male,Mcphersonview,2024-01-07,2025-01-06,Monthly,Visa,Manual,Regular,Electronics,Horror,Smartphone,Medium,3.8,7
2,3,Jonathan Parrish,brooke16@example.org,brooke16,1994-12-06,Female,Youngfort,2024-04-13,2025-04-13,Monthly,Mastercard,Manual,Regular,Books,Comedy,Smart TV,Low,3.3,8
3,4,Megan Williams,elizabeth31@example.net,elizabeth31,1964-12-22,Female,Feliciashire,2024-01-24,2025-01-23,Monthly,Amex,Auto-renew,Regular,Electronics,Documentary,Smart TV,High,3.3,7
4,5,Kathryn Brown,pattersonalexandra@example.org,pattersonalexandra,1961-06-04,Male,Port Deborah,2024-02-14,2025-02-13,Annual,Visa,Auto-renew,Frequent,Clothing,Drama,Smart TV,Low,4.3,1


In [8]:
relations = graph.refresh_schema()
print(relations)

None


In [9]:
people_query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/vansh-khaneja/test5/main/amazon_prime_users.csv' AS row

// Merge Person node using 'Name' and 'Email Address'
MERGE (person:Person {name: COALESCE(row.`Name`, 'Unknown'), email: COALESCE(row.`Email Address`, 'Unknown Email')})

// Merge Location node only if 'Location' is not null
MERGE (location:Location {name: COALESCE(row.`Location`, 'Unknown Location')})

// Merge Subscription Plan node only if 'Subscription Plan' is not null
MERGE (plan:SubscriptionPlan {name: COALESCE(row.`Subscription Plan`, 'Unknown Plan')})

// Merge Device node for 'Devices Used'
MERGE (deviceNode:Device {name: COALESCE(row.`Devices Used`, 'Unknown Device')})
MERGE (person)-[:USES]->(deviceNode)

// Relationships
MERGE (person)-[:LIVES_IN]->(location)
MERGE (person)-[:SUBSCRIBED_TO]->(plan)
MERGE (person)-[:HAS_USER_ID]->(:UserID {id: COALESCE(row.`User ID`, 'Unknown ID')})
MERGE (person)-[:HAS_USERNAME]->(:Username {name: COALESCE(row.`Username`, 'Unknown Username')})
MERGE (person)-[:HAS_BIRTH_DATE]->(:BirthDate {date: COALESCE(row.`Date of Birth`, 'Unknown Date')})
MERGE (person)-[:HAS_GENDER]->(:Gender {type: COALESCE(row.`Gender`, 'Unknown Gender')})
MERGE (person)-[:MEMBERSHIP_STARTED_ON]->(:MembershipStartDate {date: COALESCE(row.`Membership Start Date`, 'Unknown Start Date')})
MERGE (person)-[:MEMBERSHIP_ENDED_ON]->(:MembershipEndDate {date: COALESCE(row.`Membership End Date`, 'Unknown End Date')})
MERGE (person)-[:HAS_PAYMENT_INFO]->(:PaymentInformation {info: COALESCE(row.`Payment Information`, 'Unknown Payment Info')})
MERGE (person)-[:HAS_RENEWAL_STATUS]->(:RenewalStatus {status: COALESCE(row.`Renewal Status`, 'Unknown Status')})
MERGE (person)-[:HAS_USAGE_FREQUENCY]->(:UsageFrequency {frequency: COALESCE(row.`Usage Frequency`, 'Unknown Frequency')})
MERGE (person)-[:HAS_PURCHASE_HISTORY]->(:PurchaseHistory {history: COALESCE(row.`Purchase History`, 'Unknown History')})
MERGE (person)-[:HAS_FAVORITE_GENRES]->(:FavoriteGenres {genres: COALESCE(row.`Favorite Genres`, 'Unknown Genres')})
MERGE (person)-[:HAS_ENGAGEMENT_METRICS]->(:EngagementMetrics {metrics: COALESCE(row.`Engagement Metrics`, 'Unknown Metrics')})
MERGE (person)-[:GAVE_FEEDBACK]->(:Feedback {ratings: COALESCE(row.`Feedback/Ratings`, 'No Feedback')})
MERGE (person)-[:INTERACTED_WITH_SUPPORT]->(:CustomerSupport {interactions: COALESCE(row.`Customer Support Interactions`, 'No Interactions')})

"""


In [10]:
graph.query(people_query)

[]

In [12]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=GROQ_API_KEY,model_name="llama-3.1-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c5bf236fb20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c5bf236ee60>, model_name='llama-3.1-70b-versatile', groq_api_key=SecretStr('**********'))

In [18]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    graph=graph, llm=llm, verbose=True, validate_cypher=True
)
ans = chain.run("Name of location where Heidi Brock lives in return with proper namings")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Heidi Brock"})-[:LIVES_IN]->(l:Location) RETURN p.name AS PersonName, l.name AS LocationName
Full Context:
[{'PersonName': 'Heidi Brock', 'LocationName': 'Lake Gloriaport'}]

> Finished chain.


In [19]:
print(ans)

Heidi Brock lives in Lake Gloriaport.
